In [1]:
import pandas as pd
from sqlalchemy import create_engine, text


In [2]:
# def db_connection(user, password, host, port,db):
#     engine= psycopg2.connect(
#          user= user,
#          password= password,
#          host= host,
#          port=port,
#          database= db
#     )
#     return engine

In [5]:
def write_connection(user, password, host, port, db):
    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')
    connection = engine.connect()
    return engine, connection


In [6]:
engine, connection = write_connection('postgres', 'postgres', 'localhost', 5432, 'retaildwh')


In [7]:
#db_engine= db_connection(user='postgres', password='postgres', host='localhost', port=5432 ,db='retaildwh')


# DimProduct

In [9]:
def Load_DimProduct(engine= connection):
    
    # Read SQL query from file
    with open('../ETL-SCD-Queries/dimproduct.sql', 'r') as file:
        sql_query = file.read()


    results = engine.execute(text(sql_query)).fetchall()
    dimproduct_latest = pd.DataFrame(results)

    dimproduct_latest['productid']= range(1,len(dimproduct_latest)+1, 1)
    dimproduct_latest['name']= 'Product-'+dimproduct_latest['stockcode']
    # dimproduct_latest['processed_date']= date.today() + timedelta(days=1)
    dimproduct_latest['processed_date']= pd.to_datetime(dimproduct_latest['processed_date'])
    
    #DWH DIM Attributes: productid name stockcode description processed_date
    dimproduct_latest= dimproduct_latest.reindex(columns=['productid', 'name', 'stockcode', 'description', 'processed_date'])
    dimproduct_latest= dimproduct_latest.set_index('productid')
    
    #append the the latest data, latest contain the data that updated and new recordes.
    dimproduct_latest.to_sql(name='dimproduct', con=engine, if_exists='replace', index=True, index_label='productid')
    #print(dimproduct_latest.head())
    
    engine.commit()
    return dimproduct_latest
df= Load_DimProduct()
df

,name,stockcode,description,processed_date
productid,,,,
1,Product-20719,20719,WOODLAND CHARLOTTE BAG,2024-02-25
2,Product-21668,21668,RED STRIPE CERAMIC DRAWER KNOB,2024-02-25
3,Product-21669,21669,BLUE STRIPE CERAMIC DRAWER KNOB,2024-02-25
4,Product-21670,21670,BLUE SPOT CERAMIC DRAWER KNOB,2024-02-25
5,Product-21671,21671,RED SPOT CERAMIC DRAWER KNOB,2024-02-25
...,...,...,...,...
71,Product-84596F,84596F,SMALL MARSHMALLOWS PINK BOWL,2024-02-25
72,Product-84836,84836,ZINC METAL HEART DECORATION,2024-02-25
73,Product-84879,84879,ASSORTED COLOUR BIRD ORNAMENT,2024-02-23


In [14]:
df= pd.read_sql(text('delete from dimproduct where processed_date="2024-02-25"'), con= connection)
df

InternalError: (psycopg2.errors.InFailedSqlTransaction) current transaction is aborted, commands ignored until end of transaction block

[SQL: delete from dimproduct where processed_date="2024-02-25"]
(Background on this error at: https://sqlalche.me/e/20/2j85)

In [17]:
df['stockcode'].count()

56

In [18]:
df[df['stockcode'].duplicated(keep=False)].sort_values('stockcode')

,name,stockcode,description,processed_date
productid,,,,


In [19]:
num_duplicates = df.duplicated(subset=['stockcode']).sum()
print("Number of duplicates in stockcode column:", num_duplicates)


Number of duplicates in stockcode column: 0


# DIM Customer

In [16]:
def load_dimcustomer(conn= connection):
    with open('../ETL-SCD-Queries/dimcustomer.sql') as file:
        query= file.read()
        
    result= conn.execute(text(query)).fetchall()
    latest_customers= pd.DataFrame(result)
    
    latest_customers['name']= 'Customer'+latest_customers['customerid'].astype(str)
    
    # customerid, country, name, processed_date
    latest_customers= latest_customers.reindex(columns=['customerid', 'country', 'name', 'processed_date'])
    latest_customers= latest_customers.set_index('customerid')
    #latest_customers.to_sql(name='dimcustomer', con=conn, if_exists='replace', index=True, index_label='customerid')
    return latest_customers
df= load_dimcustomer()
df

InternalError: (psycopg2.errors.InFailedSqlTransaction) current transaction is aborted, commands ignored until end of transaction block

[SQL: SELECT customerid, country, processed_date
FROM (
    SELECT *, 
           ROW_NUMBER() OVER(PARTITION BY customerid ORDER BY processed_date DESC) AS rn
    FROM (
        SELECT customerid, country, processed_date 
        FROM dimcustomer
        
        UNION ALL
        
        SELECT customerid, country, CURRENT_DATE + INTERVAL '1 day' AS processed_date 
        FROM retail_cleaned
    ) AS combined
) AS nt
WHERE rn = 1

]
(Background on this error at: https://sqlalche.me/e/20/2j85)

# Dim Date

In [42]:
def load_dimdate(db_connection=connection):
    try:
        old_max = """
            select max(invoicedate) as max_date from dimdate
        """
        new_max = """
            select max(invoicedate) as max_date from retail_cleaned
        """
        old_max_date_df = pd.read_sql(text(old_max), con=db_connection).copy()
        old_max_date = old_max_date_df['max_date'].iloc[0]
        new_max_date_df = pd.read_sql(text(new_max), con=db_connection)
        new_max_date = new_max_date_df['max_date'].iloc[0]
        if new_max_date > old_max_date:
            print("new is greater create a range")
            start_range = old_max_date + pd.Timedelta(days=1)
            end_range = new_max_date
            print(start_range, end_range)
            Dimdate = pd.DataFrame({'InvoiceDate': pd.date_range(start=start_range, end=end_range, freq='D')})
            Dimdate['DateKey'] = Dimdate['InvoiceDate']
            Dimdate['Date'] = Dimdate['InvoiceDate'].dt.strftime('%Y-%m-%d')
            Dimdate['Year'] = Dimdate['InvoiceDate'].dt.strftime('%Y')
            Dimdate['MonthNo'] = Dimdate['InvoiceDate'].dt.strftime('%m')
            Dimdate['MonthName'] = Dimdate['InvoiceDate'].dt.strftime('%B')
            Dimdate['Day'] = Dimdate['InvoiceDate'].dt.strftime('%d')
            Dimdate['quarter'] = Dimdate['InvoiceDate'].dt.quarter
            Dimdate.columns = Dimdate.columns.str.lower()
            Dimdate = Dimdate.reset_index(drop=True).set_index('datekey')
            print(Dimdate)
            #Dimdate.to_sql(name='dimdate', con=db_connection, if_exists='append', index=True, index_label='datekey')
            db_connection.commit()  # Commit the transaction
            print('add the range')
        else:
            print("no updates in date")
    except Exception as e:
        print("An error occurred:", e)

load_dimdate()


no updates in date


# Fact Sales

In [47]:
def Fact_Sales(conn= connection):
    
    try:
        with open('../ETL-SCD-Queries/fact_sales.sql') as file:
            query= file.read()
        result= conn.execute(text(query))
        factDF= pd.DataFrame(result)
        max_saleskey_df = pd.read_sql(text('select max(saleskey) as max_date from fact_sales'), con=conn)
    except Exception as e:
        print("Got ERROR in Fact Sales table: while Executing the SQL Script")
        
    max_saleskey_id = max_saleskey_df['max_date'].iloc[0] +1
    factDF['saleskey']= range(max_saleskey_id, len(factDF)+max_saleskey_id, 1)
    factDF= factDF.set_index('saleskey')
    try:
        factDF.to_sql(name='fact_sales', con= conn, if_exists='append', index=True, index_label='saleskey')
    except Exception as e:
        print("Got ERROR in Fact Sales table:", e)
    return factDF

df= Fact_Sales()
df


,invoiceno,datekey,customerkey,productkey,unitprice,quantity
saleskey,,,,,,
21,536373,2010-12-01 09:02:00,17850,7,4.25,6
22,536373,2010-12-01 09:02:00,17850,6,7.65,2
23,536373,2010-12-01 09:02:00,17850,5,3.39,6
24,536373,2010-12-01 09:02:00,17850,4,3.39,6
25,536373,2010-12-01 09:02:00,17850,3,2.75,8
26,536373,2010-12-01 09:02:00,17850,2,3.39,6
27,536373,2010-12-01 09:02:00,17850,1,2.55,6
28,536372,2010-12-01 09:01:00,17850,8,1.85,6
29,536372,2010-12-01 09:01:00,17850,9,1.85,6


In [28]:
connection.close()